In [1]:
import torch
import numpy as np
import random
import torch.optim as optim

from algorithms import *
from models import *
from dataloaders import *

from tabulate import tabulate

In [2]:
device = 'cpu'
dataset = SyntheticDataset(device=device)

In [3]:
def lr_fb_model_runner(dataset, hp, seeds):
    test = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    train = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    val = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}

    def append_res(l,acc,ei,dp,eo,eodd):
        l['accuracy'].append(acc)
        l['ei_disparity'].append(ei)
        l['dp_disparity'].append(dp)
        l['eo_disparity'].append(eo)
        l['eodd_disparity'].append(eodd)

    for i in range(len(seeds)):
        print('training seed', seeds[i] ,'started')
        random.seed(seeds[i])
        np.random.seed(seeds[i])
        torch.manual_seed(seeds[i]) 

        model = logReg(num_features=dataset.XZ_train.shape[1])
        model = model.to(device)
        
        lr = hp['learning_rate']
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
        
        results = trainer_fb_fair(
            model,
            dataset,
            optimizer,
            device,
            n_epochs=hp['n_epochs'],
            batch_size=hp['batch_size'], 
            z_blind=False,
            fairness=hp['fairness'], 
            lambda_=hp['lambda_'], 
            optimal_effort=True, 
            delta_effort=hp['delta_effort']
            )
        
        append_res(train,results.train_acc_hist[-1],results.train_ei_hist[-1],results.train_dp_hist[-1],results.train_eo_hist[-1],results.train_eodd_hist[-1])
        append_res(val,results.val_acc,results.val_ei,results.val_dp,results.val_eo,results.val_eodd)
        append_res(test,results.test_acc,results.test_ei,results.test_dp,results.test_eo,results.test_eodd)

    def get_res(l):
        res = {}
        res['accuracy_mean'] = np.mean(l['accuracy'])
        res['accuracy_var'] = np.std(l['accuracy'])
        res['accuracy_list'] = l['accuracy']
        res['ei_mean'] = np.mean(l['ei_disparity'])
        res['ei_var'] = np.std(l['ei_disparity'])
        res['ei_list'] = l['ei_disparity']
        res['dp_mean'] = np.mean(l['dp_disparity'])
        res['dp_var'] = np.std(l['dp_disparity'])
        res['dp_list'] = l['dp_disparity']
        res['eo_mean'] = np.mean(l['eo_disparity'])
        res['eo_var'] = np.std(l['eo_disparity'])
        res['eo_list'] = l['eo_disparity']
        res['eodd_mean'] = np.mean(l['eodd_disparity'])
        res['eodd_var'] = np.std(l['eodd_disparity'])
        res['eodd_list'] = l['eodd_disparity']
        return res

    res_train = get_res(train)
    res_val = get_res(val)
    res_test = get_res(test)
    print('Training finished for all seeds.')
    
    return res_train, res_val, res_test


def experiment_runner(dataset, SGD_hp, EI_hp, DP_hp, EO_hp, EODD_hp, seeds):
    
    _, _, SGD = lr_fb_model_runner(dataset, SGD_hp, seeds)
    _, _, EI = lr_fb_model_runner(dataset, EI_hp, seeds)
    _, _, DP = lr_fb_model_runner(dataset, DP_hp, seeds)
    _, _, EO = lr_fb_model_runner(dataset, EO_hp, seeds)
    _, _, EODD = lr_fb_model_runner(dataset, EODD_hp, seeds)
    
    return SGD, EI, DP, EO, EODD

def hyperparameter_test(dataset, hp_test, seed=0):
    hp = hp_test.copy()
    result = []
    for i in hp_test['learning_rate']:
        for k in hp_test['lambda_']:
            c = []
            hp['learning_rate'] = i
            hp['lambda_'] = k
            try:
                train, val, _ = lr_fb_model_runner(dataset, hp, seeds=[seed])
            except:
                continue
            c.append(hp['learning_rate'])
            c.append(hp['lambda_'])
            c.append(train['accuracy_mean'])
            c.append(val['accuracy_mean'])
            c.append(val['ei_mean'])
            c.append(val['dp_mean'])
            c.append(val['eo_mean'])
            c.append(val['eodd_mean'])
            result.append(c)
    print(tabulate(result, headers=['learning_rate', 'lambda_', 'accuracy_train', 'accuracy_val','ei', 'dp', 'eo', 'eodd']))

# SGD Hyperparameter Selection

In [4]:
SGD_hp_test = {}
SGD_hp_test['learning_rate'] = [0.0001, 0.001, 0.01, 0.1]
SGD_hp_test['lambda_'] = [0]
SGD_hp_test['n_epochs'] = 100
SGD_hp_test['batch_size'] = 1024
SGD_hp_test['fairness'] = ''
SGD_hp_test['delta_effort'] = 0.5

hyperparameter_test(dataset, SGD_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 100/100 [00:08<00:00, 11.55epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:08<00:00, 11.79epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:08<00:00, 11.95epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:08<00:00, 11.62epochs/s]


Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val        ei         dp          eo        eodd
---------------  ---------  ----------------  --------------  --------  ---------  ----------  ----------
         0.0001          0          0.794219        0.798438  0.175972  0.0568639  0.359984    0.359984
         0.001           0          0.972187        0.969063  0.197906  0.121821   0.00340553  0.0500687
         0.01            0          0.997188        0.9975    0.135307  0.101419   0.00158604  0.00414326
         0.1             0          0.997969        0.998125  0.114007  0.100794   0.00158604  0.00276217


In [5]:
SGD_hp = SGD_hp_test.copy()
SGD_hp['learning_rate'] = 0.1
SGD_hp['lambda_'] = 0

# EI Hyperparameter Selection

In [6]:
EI_hp_test = SGD_hp_test.copy()
EI_hp_test['learning_rate'] = [1e-3, 1e-2]
EI_hp_test['lambda_'] = [0, 0.2, 0.5, 0.8, 1, 2, 10]
EI_hp_test['batch_size'] = 128
EI_hp_test['fairness'] = 'EI'

hyperparameter_test(dataset, EI_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 100/100 [00:15<00:00,  6.48epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:15<00:00,  6.49epochs/s]


Training finished for all seeds.
training seed 0 started


...

Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:15<00:00,  6.65epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:15<00:00,  6.32epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val          ei          dp           eo         eodd
---------------  ---------  ----------------  --------------  ----------  ----------  -----------  -----------
          0.001        0            0.9975          0.9975    0.130242    0.101419    0.00158604   0.00414326
          0.001        0.2          0.997188        0.996875  0.00592022  0.0976471   0.00555115   0.00555115
          0.001        0.5          0.994141        0.994375  0.00662543  0.0945332   0.0134814    0.0134814
          0.001        0.8          0.785547        0.787813  0.199056    0.112018    0.538462     0.538462
          0.001        1            0.400781        0.395937  0           0.00123887  0            0.00293698
          0.001        2            0.601406        0.605938  0           0           0            0
          0.001       10            0.601406        0.605938  0           0           0          

In [7]:
EI_hp = EI_hp_test.copy()
EI_hp['learning_rate'] = 0.01
EI_hp['lambda_'] = 0.2

# DP Hyperparameter Selection

In [ ]:
DP_hp_test = SGD_hp_test.copy()
DP_hp_test['learning_rate'] = [1e-2, 1e-1]
DP_hp_test['lambda_'] = [0, 0.01, 0.1, 0.2, 0.5, 0.8]
DP_hp_test['batch_size'] = 128
DP_hp_test['fairness'] = 'DP'

hyperparameter_test(dataset, DP_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 100/100 [00:11<00:00,  8.52epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:11<00:00,  8.81epochs/s]


Training finished for all seeds.
training seed 0 started


...

Training: 100%|██████████| 100/100 [00:11<00:00,  8.93epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:11<00:00,  9.01epochs/s]


Training finished for all seeds.
training seed 0 started


Training:  42%|████▏     | 42/100 [00:04<00:06,  8.94epochs/s]

In [9]:
DP_hp = DP_hp_test.copy()
DP_hp['learning_rate'] = 0.01
DP_hp['lambda_'] = 0.5

# EO Hyperparameter Selection

In [10]:
EO_hp_test = SGD_hp_test.copy()
EO_hp_test['learning_rate'] = [1e-3, 1e-2]
EO_hp_test['lambda_'] = [0, 0.2, 0.5, 0.8, 1, 2, 10]
EO_hp_test['batch_size'] = 128
EO_hp_test['fairness'] = 'EO'

hyperparameter_test(dataset, EO_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 100/100 [00:12<00:00,  8.10epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:12<00:00,  7.86epochs/s]


Training finished for all seeds.
training seed 0 started


...

Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:12<00:00,  8.15epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:12<00:00,  8.13epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val         ei         dp           eo        eodd
---------------  ---------  ----------------  --------------  ---------  ---------  -----------  ----------
          0.001        0           0.9975          0.9975     0.130242   0.101419   0.00158604   0.00414326
          0.001        0.2         0.996719        0.996875   0.161248   0.102669   0.000793021  0.00621489
          0.001        0.5         0.994453        0.994687   0.201905   0.105482   0            0.0117392
          0.001        0.8         0.985781        0.985      0.298939   0.115169   0            0.0331461
          0.001        1           0.398594        0.394062   0          0          0            0
          0.001        2           0.371797        0.3675     0.0117647  0.0087508  0.00680076   0.00680076
          0.001       10           0.0973438       0.09125    0.0298112  0.0813271  0.00147619   0.00690543
      

In [ ]:
EO_hp = EO_hp_test.copy()
EO_hp['learning_rate'] = 0.001
EO_hp['lambda_'] = 0.5

# EOD Hyperparameter Selection

In [11]:
EOD_hp_test = SGD_hp_test.copy()
EOD_hp_test['learning_rate'] = [1e-3, 1e-2]
EOD_hp_test['lambda_'] = [0, 0.2, 0.5, 0.8, 1, 2, 10]
EOD_hp_test['batch_size'] = 128
EOD_hp_test['fairness'] = 'EOD'

hyperparameter_test(dataset, EOD_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 100/100 [00:08<00:00, 11.43epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:09<00:00, 11.07epochs/s]


Training finished for all seeds.
training seed 0 started


...

Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:08<00:00, 11.45epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:08<00:00, 11.46epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val         ei        dp           eo        eodd
---------------  ---------  ----------------  --------------  ---------  --------  -----------  ----------
          0.001        0          0.9975           0.9975     0.130242   0.101419  0.00158604   0.00414326
          0.001        0.2        0.9975           0.9975     0.130242   0.101419  0.00158604   0.00414326
          0.001        0.5        0.997422         0.9975     0.130937   0.101419  0.00158604   0.00414326
          0.001        0.8        0.997031         0.997812   0.136742   0.101732  0.000793021  0.00414326
          0.001        1          0.601406         0.605938   0          0         0            0
          0.001        2          0.263125         0.273438   0.0360757  0.302996  0            0.470329
          0.001       10          0.263516         0.274062   0.0380477  0.302349  0            0.469297
          0.01   

In [ ]:
EOD_hp = EOD_hp_test.copy()
EOD_hp['learning_rate'] = 0.01
EOD_hp['lambda_'] = 0.2